In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# logistic regression
from sklearn.linear_model import LogisticRegression

# decision tree ;)
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

# random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

Install Kaggle Library:

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c caltech-cs155-2022-mp1

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


Download Datasets:

In [ ]:
! unzip LOANS_TRAIN.csv.zip
! unzip LOANS_TEST.csv.zip
train_df = pd.read_csv('/content/LOANS_TRAIN.csv')
test_df = pd.read_csv('/content/LOANS_TEST.csv')

Archive:  LOANS_TRAIN.csv.zip
  inflating: LOANS_TRAIN.csv         
Archive:  LOANS_TEST.csv.zip
  inflating: LOANS_TEST.csv          


In [ ]:
display(train_df)

Find all NaNs:

In [ ]:
train_df.isna().sum()

In [ ]:
#handle NaN
train_df = train_df.fillna(0)
display(train_df)
test_df = test_df.fillna(0)

,id,loan_amnt,term_(months),int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,title,zip_code,addr_state,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,loan_status
0,0,5000,36,10.65%,162.87,B,B2,0,10+ years,RENT,24000.0,Verified,Dec-2011,credit_card,Computer,860xx,AZ,27.65,Jan-1985,3,0,13648,83.7%,9,f,Individual,0.0,0.0,Fully Paid
1,1,2500,60,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,car,bike,309xx,GA,1.00,Apr-1999,3,0,1687,9.4%,4,f,Individual,0.0,0.0,Charged Off
2,2,2400,36,15.96%,84.33,C,C5,0,10+ years,RENT,12252.0,Not Verified,Dec-2011,small_business,real estate business,606xx,IL,8.72,Nov-2001,2,0,2956,98.5%,10,f,Individual,0.0,0.0,Fully Paid
3,3,10000,36,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,other,personel,917xx,CA,20.00,Feb-1996,10,0,5598,21%,37,f,Individual,0.0,0.0,Fully Paid
4,4,3000,60,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,other,Personal,972xx,OR,17.94,Jan-1996,15,0,27783,53.9%,38,f,Individual,0.0,0.0,Fully Paid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197244,199994,12000,36,14.09%,410.66,B,B5,security service,1 year,MORTGAGE,45000.0,Not Verified,Sep-2012,debt_consolidation,bill consolidation,782xx,TX,23.12,Jun-2000,7,0,5912,81%,15,f,Individual,1.0,0.0,Fully Paid
197245,199995,1500,36,7.90%,46.94,A,A4,USAF,5 years,RENT,42000.0,Source Verified,Sep-2012,other,Auto Repairs,890xx,NV,18.23,Aug-2005,5,0,3597,29.5%,10,f,Individual,0.0,0.0,Fully Paid
197247,199997,11000,36,14.09%,376.44,B,B5,Gateway Regional Medical Center,2 years,RENT,45000.0,Not Verified,Sep-2012,debt_consolidation,Debt Consolidation,631xx,MO,8.91,Oct-2006,18,0,8956,43.7%,20,f,Individual,0.0,0.0,Fully Paid
197248,199998,31400,60,22.95%,884.28,F,F1,Navy Federal Credit Union,3 years,MORTGAGE,85000.0,Verified,Sep-2012,debt_consolidation,debt free plan!,220xx,VA,24.76,Oct-1999,12,0,23150,77.4%,23,f,Individual,4.0,0.0,Charged Off


In [ ]:
train_data = train_df.to_numpy()
test_data = test_df.to_numpy()

N, D = train_data.shape

X_train = train_data[:,1:-1]
Y_train = [1 if train_data[i,-1] == 'Fully Paid' else 0 for i in range(N)]

X_test = test_data[:,1:]

print(X_train[0])
print(X_test[0])

[5000 36 ' 10.65%' 162.87 'B' 'B2' 0 '10+ years' 'RENT' 24000.0 'Verified'
 'Dec-2011' 'credit_card' 'Computer' '860xx' 'AZ' 27.65 'Jan-1985' 3 0
 13648 '83.7%' 9 'f' 'Individual' 0.0 0.0]
[4975.0 36 '  6.03%' 151.42 'A' 'A1'
 'XTRAC, LLC a Fidelity Investments Compan' '1 year' 'MORTGAGE' 67000.0
 'Not Verified' 'Mar-2012' 'debt_consolidation' 'Debt Consolidation'
 '840xx' 'UT' 1.34 'Jul-1990' 7.0 0.0 1075.0 '8.5%' 29.0 'f' 'Individual'
 0.0 0.0]


In [ ]:
print(X_test[0])

In [ ]:
titles = set(X_train[:,12])
print(len(titles))
print(N)

In [ ]:
# convert percents to floats. Run once for every time we run cell above
# remove xx from zip codes
def float_percent(x):
    return float(x.strip('%'))/100

for i in range(N):
    X_train[i][2] = float_percent(str(X_train[i][2]))
    X_train[i][21] = float_percent(str(X_train[i][21]))
    X_train[i][14] = int(X_train[i][14].strip('xx'))

print(X_train[0])

In [ ]:
# convert grade and subgrade to numerics
def int_grade(grade):
    if type(grade) is str and len(grade) == 1:
        return int(ord(grade) - ord('A'))
    return -1

def float_subgrade(subgrade):
    if type(subgrade) is str and len(subgrade) == 2:
        grade = subgrade[0]
        sub = subgrade[1]
        x = 0
        x += int_grade(grade)
        x += int(sub) * 0.1
        return x
    return -1

#for i in range(N):
    #X_train[i][4] = int_grade(X_train[i][4])
    #X_train[i][5] = float_subgrade(X_train[i][5])
#print(X_train[0])

In [ ]:
def int_emplength(x):
    if x == '< 1 year' or x == 0:
        return 0
    nums = []
    for c in x:
        if c.isdigit():
            nums.append(c)
    num = ''.join(nums)
    if num == '':
        return 0
    return int(num)

#for i in range(N):
    #X_train[i][7] = int_emplength(X_train[i][7])
print(X_train[0])

In [ ]:
def int_verificationstatus(x):
  if x == 'Not Verified':
    return 0
  elif x == 'Verified':
    return 1
  elif x == 'Source Verified':
    return 2
  return -1

for i in range(N):
  X_train[i][10] = int_verificationstatus(X_train[i][10])
print(X_train[0])

In [ ]:
def int_homeownership(x):
  if x == 'MORTGAGE':
    return 0
  elif x == 'RENT':
    return 1
  elif x == 'OWN':
    return 2
  return -1

for i in range(N):
  X_train[i][8] = int_homeownership(X_train[i][8])

print(X_train[0])

In [ ]:
def int_initialliststatus(x):
  if x == 'f':
    return 1
  return 0

for i in range(N):
  X_train[i][23] = int_initialliststatus(X_train[i][23])

print(X_train[0])

In [ ]:
def int_applicationtype(x):
  if x == 'Individual':
    return 1
  return 0

for i in range(N):
  X_train[i][24] = int_applicationtype(X_train[i][24])

print(X_train[0])

In [ ]:
state_dict = {'AL':1, 'AK':2, 'AZ':3, 'AR':4, 'CA':5, 'CO':6, 'CT':7, 'DE':8, 'FL':9, 'GA':10,
              'HI':11, 'ID':12, 'IL':13, 'IN':14, 'IA':15, 'KS':16, 'KY':17, 'LA':18, 'ME':19, 'MD':20, 
              'MA':21, 'MI':22, 'MN':23, 'MS':24, 'MO':25, 'MT':26, 'NE':27, 'NV':28, 'NH':29, 'NJ':30, 
              'NM':31, 'NY':32, 'NC':33, 'ND':34, 'OH':35, 'OK':36, 'OR':37, 'PA':38, 'RI':39, 'SC':40,
              'SD':41, 'TN':42, 'TX':43, 'UT':44, 'VT':45, 'VA':46, 'WA':47, 'WV':48, 'WI':49, 'WY':50}

def int_addrstate(x):
  if x in state_dict:
    return state_dict[x]
  return -1

for i in range(N):
  X_train[i][15] = int_addrstate(X_train[i][15])

print(X_train[0])

In [ ]:
purpose_dict = {'other':1, 'moving':2, 'house':3, 'wedding':4, 'car':5, 'credit_card':6, 'home_improvement':7, 
               'renewable_energy':8, 'small_business':9, 'debt_consolidation':10, 'medical':11, 
               'educational':12, 'major_purchase':13, 'vacation':14}

def int_purpose(x):
  if x in purpose_dict:
    return purpose_dict[x]
  return -1

for i in range(N):
  X_train[i][12] = int_purpose(X_train[i][12])

print(X_train[0])

In [ ]:
months = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 
          'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
def month_year(mt):
    d = mt.split('-')
    return months[d[0]], int(d[1])

X_copy = X_train.copy()
ms = []
ys = []
for i in range(len(X_copy)):
    m,y = month_year(X_copy[i][17])
    ms.append(m)
    ys.append(y)
    
test = np.zeros((N,D-1)).astype('object')
test[:,:17] = X_copy[:,:17]
test[:,17] = ms
test[:,18] = ys
test[:,19:] = X_copy[:,18:]
print(test[0])

ms2 = []
ys2 = []
for i in range(len(test)):
    m,y = month_year(test[i][11])
    ms2.append(m)
    ys2.append(y)

test2 = np.zeros((N,D)).astype('object')
test2[:,:11] = test[:,:11]
test2[:,11] = ms2
test2[:,12] = ys2
test2[:,13:] = test[:,12:]
print(test2[0])
    

In [ ]:
dick = {}
# Generic words that apply to multiple categories: 
# Center (health center, care center, medical center)
# County (just wack, other categories or other/precedence more important)
trash = ['the', 'and', '&', 'of', 'inc', 'inc.', 'center', 'county']

for i in range(len(X_train)):
  title = X_train[i][6]
  if not isinstance(title, int):
    title = title.lower()
    title_split = str(title).split()
    for word in title_split:
      if word not in dick.keys():
        dick[word] = 1
      else:
        dick[word] += 1


top_ten = []
for element in dick:
  if element in trash:
    continue
  freq = dick[element]
  if len(top_ten) < 30:
    top_ten.append((freq, element))
  else:
    top_ten.sort()
    if top_ten[0][0] < freq:
      top_ten[0] = (freq, element)

slatt = 0
for item in top_ten:
  print(item)
  #slatt += item[0]

print(slatt)


In [ ]:
# Running order of precedence:
# 

for i in range(len(X_train)):
  title = X_train[i][6]
  if not isinstance(title, int):
    title = title.lower()
    if 'services' in title:
      print(title)

## Process all data:

In [ ]:
def process2(X):
    N, D = X.shape
    for i in range(len(X)):
        X[i][2] = float_percent(str(X[i][2]))
        X[i][4] = int_grade(X[i][4])
        X[i][5] = float_subgrade(X[i][5])
        X[i][7] = int_emplength(X[i][7])
        X[i][8] = int_homeownership(X[i][8])
        X[i][10] = int_verificationstatus(X[i][10])
        #X[i][12] = int_purpose(X[i][12])
        #X[i][14] = int(X[i][14].strip('xx'))
        #X[i][15] = int_addrstate(X[i][15])
        X[i][21] = float_percent(str(X[i][21]))
        #X[i][23] = int_initialliststatus(X[i][23])
        #X[i][24] = int_applicationtype(X[i][24])
    
    X_copy = X.copy()
    m_list = []
    y_list = []
    for i in range(len(X_copy)):
        m,y = month_year(X_copy[i][17])
        m_list.append(m)
        y_list.append(y)
    
    t = np.zeros((N,D+1)).astype('object')
    t[:,:17] = X_copy[:,:17]
    t[:,17] = m_list
    t[:,18] = y_list
    t[:,19:] = X_copy[:,18:]

    m_list2 = []
    y_list2 = []
    for i in range(len(t)):
        m,y = month_year(t[i][11])
        m_list2.append(m)
        y_list2.append(y)

    t2 = np.zeros((N,D+2)).astype('object')
    t2[:,:11] = t[:,:11]
    t2[:,11] = m_list2
    t2[:,12] = y_list2
    t2[:,13:] = t[:,12:]
    print(t2[6])

    t3 = np.delete(t2, [6,13,14,15,16,25,26], axis=1)

    return t3

In [ ]:
X_train_copy = X_train.copy()
X_train_processed = process2(X_train_copy)
X_test_copy = X_test.copy()
X_test_processed = process2(X_test_copy)
print(X_train_processed[6])

[7000 60 0.15960000000000002 170.08 2 2.5 'Southern Star Photography' 8 1
 47004.0 0 12 2011 'debt_consolidation' 'Loan' '280xx' 'NC' 23.51 7 2005 7
 0 17726 0.856 11 'f' 'Individual' 0.0 0.0]
[21000.0 60 0.1367 485.05 1 1.5 'Jefferson County Public Works' 10 0
 42644.04 1 2 2012 'debt_consolidation' 'payoff credit' '630xx' 'MO' 14.74
 9 1997 15.0 0.0 19435.0 0.226 31.0 'f' 'Individual' 0.0 0.0]
[7000 60 0.15960000000000002 170.08 2 2.5 8 1 47004.0 0 12 2011 23.51 7
 2005 7 0 17726 0.856 11 0.0 0.0]


In [ ]:
# Enjoy Nerds

train_df = pd.read_csv('/content/LOANS_TRAIN.csv')
train_df = train_df.dropna(subset=['emp_length'])
print(len(train_df))
train_data = train_df.to_numpy()
X_train = train_data[:,1:-1]


def float_percent(x):
    return float(x.strip('%'))/100



# Ordinal with A = 7, G = 1 (Assumption A = best)
def convert_grade_to_numeric(grade):
  return 7 - int(ord(grade) - ord('A'))

# Ordinal with A = 7, G = 1 (Assumption A1 = best)
def convert_subgrade_to_numeric(grade):
  tier = convert_grade_to_numeric(grade[0])
  number = int(grade[1])
  return (tier * 5) - number + 1

def one_hot_encode_home_ownership(X):
  home_ownership_rent = []
  home_ownership_mortgage = []
  home_ownership_own = []
  for i in range(len(X)):
    if X[i][8] == 'RENT':
      home_ownership_rent.append(1)
      home_ownership_mortgage.append(0)
      home_ownership_own.append(0)
    elif X[i][8] == 'MORTGAGE':
      home_ownership_rent.append(0)
      home_ownership_mortgage.append(1)
      home_ownership_own.append(0)
    elif X[i][8] == 'OWN':
      home_ownership_rent.append(0)
      home_ownership_mortgage.append(0)
      home_ownership_own.append(1)
    else:
      home_ownership_rent.append(0)
      home_ownership_mortgage.append(0)
      home_ownership_own.append(0)


  return home_ownership_rent, home_ownership_mortgage, home_ownership_own

def one_hot_encode_verification_status(X):
  verification_status_verified = []
  verification_status_source = []
  verification_status_not_verified = []
  for i in range(len(X)):
    if X[i][10] == 'Verified':
      verification_status_verified.append(1)
      verification_status_source.append(0)
      verification_status_not_verified.append(0)
    elif X[i][10] == 'Source Verified':
      verification_status_verified.append(0)
      verification_status_source.append(1)
      verification_status_not_verified.append(0)
    elif X[i][10] == 'Not Verified':
      verification_status_verified.append(0)
      verification_status_source.append(0)
      verification_status_not_verified.append(1)

  return verification_status_verified, verification_status_source, verification_status_not_verified


def one_hot_encode_initial_list_status(X):
  f = []
  w = []
  for i in range(len(X)):
    if X[i][23] == 'f':
      f.append(1)
      w.append(0)
    elif X[i][23] == 'w':
      f.append(0)
      w.append(1)
  return f, w



loan_amount = []
term = []
int_rate = []
installment = []
grade = []
subgrade = []
emp_len = []
annual_inc = []
purpose = []
dti = []
open_acc = []
pub_rec = []
revol_bal = []
revol_util = []
total_acc = []
pub_rec_bank = []
loan_status = []

home_ownership_rent, home_ownership_mortgage, home_ownership_own = one_hot_encode_home_ownership(X_train)
verification_status_verified, verification_status_source, verification_status_not_verified = one_hot_encode_verification_status(X_train)
initial_status_f, initial_status_w = one_hot_encode_initial_list_status(X_train)


for i in range(len(X_train)):
  loan_amount.append(X_train[i][0])
  term.append(X_train[i][1])
  int_rate.append(float_percent(str(X_train[i][2])))
  installment.append(X_train[i][3])
  grade.append(convert_grade_to_numeric(X_train[i][4]))
  subgrade.append(convert_subgrade_to_numeric(X_train[i][5]))
  emp_len.append(int_emplength(X_train[i][7]))
  annual_inc.append(X_train[i][9])
  dti.append(X_train[i][16])
  open_acc.append(X_train[i][18])
  pub_rec.append(X_train[i][19])
  revol_bal.append(X_train[i][20])
  revol_util.append(float_percent(str(X_train[1][21])))
  total_acc.append(X_train[i][22])
  pub_rec_bank.append(X_train[i][26])


loan_status = [1 if train_data[i,-1] == 'Fully Paid' else 0 for i in range(train_data.shape[0])]


data = {'loan_amnt': loan_amount, 
        'term (months)': term,
        'int_rate': int_rate,
        'installment': installment,
        'grade': grade,
        'subgrade': subgrade,
        'emp_length': emp_len,
        'home_ownership_rent': home_ownership_rent,
        'home_ownership_mortgage': home_ownership_mortgage,
        'home_ownership_own': home_ownership_own,
        'annual_inc': annual_inc,
        'verification_status_verified': verification_status_verified, 
        'verification_status_source': verification_status_source, 
        'verification_status_not_verified': verification_status_not_verified,
        'dti': dti,
        'open_acc': open_acc,
        'pub_rec': pub_rec,
        'revol_bal': revol_bal,
        'revol_util': revol_util,
        'pub_rec_bank': pub_rec_bank,
        'loan_status': loan_status
      }

# Good dataframe
# Sorry didnt generalize for X_test 
df = pd.DataFrame(data=data)
df.to_csv('cleaned_training_data.csv')




189334


In [ ]:
unique = []
other_count = 0
none_count = 0

for i in range(len(X_train)):
  grade = (X_train[i][8])
  # if grade not in unique:
  #   unique.append(grade)
  if grade == 'NONE':
    none_count += 1
  elif grade == 'OTHER':
    other_count += 1

# unique.sort()
# print(len(unique))
# print(unique)

print(none_count)
print(other_count)

# for item in data:
#   print(len(data[item]))


NameError: ignored

## Validation:

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
# 0/1 loss
def loss(preds, y):
    return float(len(np.where(preds != y)[0])) / len(y)

In [ ]:
n_folds = 5
kf = KFold(n_splits=n_folds)
train_losses = []
valid_losses = []
Y_train = np.array(Y_train).astype(int)

estimators = np.linspace(50,200,7)
for n_estimators in estimators:
    train_loss = 0
    valid_loss = 0
    for train_idx, valid_idx in kf.split(X_train_processed):
        x_train, x_valid = X_train_processed[train_idx], X_train_processed[valid_idx]
        y_train, y_valid = Y_train[train_idx], Y_train[valid_idx]
        clf = AdaBoostClassifier(n_estimators=int(n_estimators))
        clf.fit(x_train, y_train)

        train_preds = clf.predict(x_train)
        valid_preds = clf.predict(x_valid)

        train_loss += loss(train_preds, y_train)
        valid_loss += loss(valid_preds, y_valid)

    train_losses.append(train_loss / n_folds)
    valid_losses.append(valid_loss / n_folds)
    print(valid_loss/n_folds)

print(train_losses)
print(valid_losses)

KeyboardInterrupt: ignored

In [ ]:
plt.figure()
plt.plot(estimators,train_losses,label='training loss')
plt.plot(estimators,valid_losses,label='validation loss')
plt.xlabel('n_estimators')
plt.ylabel('0/1 loss')
plt.legend(loc='upper right')
plt.show()
print(estimators[np.argmin(valid_losses)])

In [ ]:
def process(X):
    for i in range(len(X)):
        X[i][2] = float_percent(str(X[i][2]))
        X[i][4] = int_grade(X[i][4])
        X[i][5] = float_subgrade(X[i][5])
        X[i][7] = int_emplength(X[i][7])
        X[i][8] = int_homeownership(X[i][8])
        X[i][10] = int_verificationstatus(X[i][10])
        X[i][12] = int_purpose(X[i][12])
        X[i][14] = int(X[i][14].strip('xx'))
        X[i][15] = int_addrstate(X[i][15])
        X[i][21] = float_percent(str(X[i][21]))
        X[i][23] = int_initialliststatus(X[i][23])
        X[i][24] = int_applicationtype(X[i][24])
    return X

X_train2 = X_train.copy()
X_train2 = process(X_train2)

In [ ]:
print(X_train2[6])

# Split up MM-YYYY
X_copy2 = X_train2.copy()
m_list = []
y_list = []
for i in range(len(X_copy2)):
    m,y = month_year(X_copy2[i][17])
    m_list.append(m)
    y_list.append(y)
    
t = np.zeros((N,D-1)).astype('object')
t[:,:17] = X_copy2[:,:17]
t[:,17] = m_list
t[:,18] = y_list
t[:,19:] = X_copy2[:,18:]
print(t[0])

m_list2 = []
y_list2 = []
for i in range(len(test)):
    m,y = month_year(test[i][11])
    m_list2.append(m)
    y_list2.append(y)

t2 = np.zeros((N,D)).astype('object')
t2[:,:11] = t[:,:11]
t2[:,11] = m_list2
t2[:,12] = y_list2
t2[:,13:] = t[:,12:]

[7000 60 0.15960000000000002 170.08 2 2.5 'Southern Star Photography' 8 1
 47004.0 0 'Dec-2011' 10 'Loan' 280 33 23.51 'Jul-2005' 7 0 17726 0.856 11
 1 0 0.0 0.0]
[5000 36 0.1065 162.87 1 1.2 0 10 1 24000.0 1 'Dec-2011' 6 'Computer' 860
 3 27.65 1 1985 3 0 13648 0.8370000000000001 9 1 0 0.0 0.0]


In [ ]:
print(t2[6])

[7000 60 0.15960000000000002 170.08 2 2.5 'Southern Star Photography' 8 1
 47004.0 0 12 2011 10 'Loan' 280 33 23.51 7 2005 7 0 17726 0.856 11 1 0
 0.0 0.0]


In [ ]:
t3 = t2.copy()
t3 = np.delete(t3, [6,14], axis=1)
print(t3[6])

[7000 60 0.15960000000000002 170.08 2 2.5 8 1 47004.0 0 12 2011 10 280 33
 23.51 7 2005 7 0 17726 0.856 11 1 0 0.0 0.0]


In [ ]:
X_test2 = X_test.copy()
X_test2 = process(X_test2)

X_c = X_test2.copy()
m_l = []
y_l = []
for i in range(len(X_c)):
    m,y = month_year(X_c[i][17])
    m_l.append(m)
    y_l.append(y)
    
testdata = np.zeros((len(X_c),D-1)).astype('object')
testdata[:,:17] = X_c[:,:17]
testdata[:,17] = m_l
testdata[:,18] = y_l
testdata[:,19:] = X_c[:,18:]
print(testdata[0])

m_l2 = []
y_l2 = []
for i in range(len(testdata)):
    m,y = month_year(testdata[i][11])
    m_l2.append(m)
    y_l2.append(y)

testdata2 = np.zeros((len(X_c),D)).astype('object')
testdata2[:,:11] = testdata[:,:11]
testdata2[:,11] = m_l2
testdata2[:,12] = y_l2
testdata2[:,13:] = testdata[:,12:]
print(testdata2)

testdata3 = testdata2.copy()
testdata3 = np.delete(testdata3, [6,14], axis=1)
print(testdata3[6])

[4975.0 36 0.0603 151.42 0 0.1 'XTRAC, LLC a Fidelity Investments Compan'
 1 0 67000.0 0 'Mar-2012' 10 'Debt Consolidation' 840 44 1.34 7 1990 7.0
 0.0 1075.0 0.085 29.0 1 0 0.0 0.0]
[[4975.0 36 0.0603 ... 0 0.0 0.0]
 [5000.0 36 0.18059999999999998 ... 0 0.0 0.0]
 [10000.0 36 0.0603 ... 0 0.0 0.0]
 ...
 [30000.0 60 0.21969999999999998 ... 0 0.0 0.0]
 [14825.0 36 0.12119999999999999 ... 0 0.0 0.0]
 [25000.0 60 0.1408 ... 0 2.0 0.0]]
[21000.0 60 0.1367 485.05 1 1.5 10 0 42644.04 1 2 2012 10 630 25 14.74 9
 1997 15.0 0.0 19435.0 0.226 31.0 1 0 0.0 0.0]


In [ ]:
clf = RandomForestClassifier()
clf.fit(t3, Y_train)

RandomForestClassifier()

In [ ]:
preds_train = clf.predict(t3)
print(sum(preds_train == Y_train)/len(Y_train))

0.9999949302915082


In [ ]:
preds = clf.predict(testdata3)
print(preds)
preds_df = pd.DataFrame(preds)
preds_df.to_csv('/content/submission.csv')

[1 1 1 ... 1 1 1]


In [ ]:
# try: state & purpose --> correlation 

def try_state():
    state_dic = {}
    tot_state_dic = {}
    leng = len(train_df)

    for k in range(leng):
      state = train_df.iloc[k]["addr_state"]
      if (train_df.iloc[k]["loan_status"] == "Fully Paid"):
        if state not in state_dic:
          state_dic[state] = 1
        else:
          state_dic[state] += 1
      if state not in tot_state_dic:
        tot_state_dic[state] = 1
      else:
        tot_state_dic[state] += 1

    proportion_dic = {}

    for k,v in state_dic.items():
        proportion = float(v) /float(tot_state_dic[k])
        proportion_dic[k] = proportion


    return proportion_dic



def try_purpose():
    purpose_dic = {}
    tot_purpose_dic = {}
    leng = len(train_df)

    for k in range(leng):
      purpose = train_df.iloc[k]["purpose"]
      if (train_df.iloc[k]["loan_status"] == "Fully Paid"):
        if purpose not in purpose_dic:
          purpose_dic[purpose] = 1
        else:
          purpose_dic[purpose] += 1
      if purpose not in tot_purpose_dic:
        tot_purpose_dic[purpose] = 1
      else:
        tot_purpose_dic[purpose] += 1

    proportion_dic = {}

    for k,v in purpose_dic.items():
        proportion = float(v) /float(tot_purpose_dic[k])
        proportion_dic[k] = proportion


    return proportion_dic

print(try_purpose())


train_df = train_df


{'car': 0.8948326055312955,
 'credit_card': 0.8746609458992948,
 'debt_consolidation': 0.8399179066187789,
 'educational': 0.8359621451104101,
 'home_improvement': 0.8691089837997055,
 'house': 0.8494242692648362,
 'major_purchase': 0.8848663439275942,
 'medical': 0.8331428571428572,
 'moving': 0.8414539829853055,
 'other': 0.8228422475381347,
 'renewable_energy': 0.838150289017341,
 'small_business': 0.7445109780439122,
 'vacation': 0.8521478521478522,
 'wedding': 0.8779369627507163}